In [9]:
from sklearn.datasets import make_regression
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [11]:
X, y = make_regression(n_samples=500, n_features=4, n_informative=4, noise=15, random_state=41)
X = pd.DataFrame(X, columns=['f1', 'f2', 'f3', 'f4'])
y = pd.Series(y)

In [13]:
X

,f1,f2,f3,f4
0,-1.829565,0.936395,-0.010417,-0.014455
1,-1.073248,-0.349861,0.482738,0.682948
2,1.072434,-1.103835,2.365345,0.750906
3,-0.582461,-1.168958,0.656672,0.918268
4,1.164526,1.768251,-0.062564,0.862613
...,...,...,...,...
495,0.593223,-2.217525,0.468373,2.186467
496,-2.111618,-0.568311,-0.818308,-0.962384
497,-0.246300,-0.602856,0.103486,-1.369246
498,-0.966430,0.731898,0.641263,0.063056


In [15]:
y

0       22.344800
1      -39.297629
2      -29.892923
3      -93.813436
4      122.401950
          ...    
495    -99.461174
496    -80.648530
497    -60.448341
498     13.904797
499    -34.397604
Length: 500, dtype: float64

# Нужно написать метод .predict()

    - метод должен принять матрицу фичей и вернуть предсказания модели
    
`нужно еще дополнить матрицу фичей столбцом из 1 слева для свободного члена`

In [20]:
class MyLineReg():
    def __init__(self, n_iter=100, learning_rate=0.1, weights=None):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.weights = weights

    def __str__(self):
        return f'MyLineReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}'

    def fit(self, X: pd.DataFrame, y:pd.Series, verbose=False):
        X_copy = X.copy()               # создадим копию нашей матрицы фичей, что бы не изменить изначальный
        X_copy.insert(0, 'base', 1)     # допишем слева столбик из 1 для свободного члела
        w = np.ones(X_copy.shape[1])    # составим вектор весов, заполненный из 1

        if verbose:                   # Напишем подсчет ошибки до начала обучения
            initial_MSE = np.mean((np.dot(X_copy, w) - y)**2)
            print(f'start | loss: {initial_MSE}')
            
        for i in range(self.n_iter): # Напишем цикл обучения
            pred = np.dot(X_copy, w) # cчитаем предасказания модели 
            error = pred - y         # вычисляем ошибку (предсказания - реальные значения)
            grad = (2/len(X_copy)) * np.dot(error, X_copy) # вычисляем градиенты по весам
            w -= self.learning_rate * grad  # обновляем веса 

            if verbose and i % 10 == 0:     # выводим логи 
                loss = np.mean(error**2)
                print(f'Iteration {i}| loss: {loss}')
        # сохроняем веса
        self.weights = w 

    def predict(self, X: pd.DataFrame)->np.array:
        X_copy = X.copy()            # делаем копию матрицы фичей, чтобы не менять изначальный датафрейм
        X_copy.insert(0, 'base', 1)  # допишем слева столбик из 1 для свободного члена
        predict = X_copy.dot(self.weights) # делаем предсказания X_copy @ self.weights
        return predict 
    
    def get_coef(self):
        return self.weights[1:] # метод для вывода весов начиная с 1го заначения
    

In [82]:
model = MyLineReg()

In [84]:
print(model)

MyLineReg class: n_iter=100, learning_rate=0.1


In [86]:
model.fit(X, y, verbose=True)

start | loss: 4843.313291528255
Iteration 0| loss: 4843.313291528255
Iteration 10| loss: 274.53456764583274
Iteration 20| loss: 226.50873707706205
Iteration 30| loss: 225.9627736593816
Iteration 40| loss: 225.95633031682144
Iteration 50| loss: 225.95625284090957
Iteration 60| loss: 225.95625189916538
Iteration 70| loss: 225.9562518876303
Iteration 80| loss: 225.95625188748812
Iteration 90| loss: 225.9562518874863


In [89]:
model.weights

array([-0.96450618, 29.48183375, 61.63596568,  4.39942233,  2.79733944])

In [91]:
model.get_coef()

array([29.48183375, 61.63596568,  4.39942233,  2.79733944])

In [93]:
X.loc[:5]

,f1,f2,f3,f4
0,-1.829565,0.936395,-0.010417,-0.014455
1,-1.073248,-0.349861,0.482738,0.682948
2,1.072434,-1.103835,2.365345,0.750906
3,-0.582461,-1.168958,0.656672,0.918268
4,1.164526,1.768251,-0.062564,0.862613
5,-0.419084,1.861414,1.102373,-0.000597


In [95]:
print(f'MSE = {np.mean((model.predict(X) - y)**2)}')
print(f'RMSE = {np.sqrt(np.mean((model.predict(X) - y)**2))}')

MSE = 225.9562518874863
RMSE = 15.031841267372613


In [97]:
y[:6]

0     22.344800
1    -39.297629
2    -29.892923
3    -93.813436
4    122.401950
5    108.061611
dtype: float64

In [99]:
model.predict(X.loc[:5]) - y[:6]

0   -19.618894
1   -10.838011
2     5.016489
3     9.084744
4    22.091544
5    -1.803300
dtype: float64

In [101]:
spid = lambda iter: 0.5 * (0.85 ** iter)

In [103]:
for i in range(10):
    print(spid(i))

0.5
0.425
0.36124999999999996
0.30706249999999996
0.26100312499999995
0.22185265624999997
0.18857475781249997
0.16028854414062496
0.13624526251953123
0.11580847314160153


In [105]:
0.5 * 0.85

0.425

In [107]:
a = X.iloc[:5]
t = y.iloc[:5]

In [109]:
a

,f1,f2,f3,f4
0,-1.829565,0.936395,-0.010417,-0.014455
1,-1.073248,-0.349861,0.482738,0.682948
2,1.072434,-1.103835,2.365345,0.750906
3,-0.582461,-1.168958,0.656672,0.918268
4,1.164526,1.768251,-0.062564,0.862613


In [111]:
t

0     22.344800
1    -39.297629
2    -29.892923
3    -93.813436
4    122.401950
dtype: float64

In [113]:
model.predict(a)

0      2.725906
1    -50.135640
2    -24.876433
3    -84.728692
4    144.493494
dtype: float64

In [116]:
from sklearn.linear_model import LinearRegression

In [118]:
mod = LinearRegression() 

In [120]:
mod.fit(X, y)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [122]:
mod.predict(a)

array([  2.7259056 , -50.13563966, -24.87643344, -84.72869209,
       144.49349451])